In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import torch
from transformers import AutoModel, AutoTokenizer

In [2]:
model_name = 'deepseek-ai/DeepSeek-OCR'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, 
                                  _attn_implementation='eager', 
                                  trust_remote_code=True, 
                                  use_safetensors=True)
model = model.eval().cuda().to(torch.bfloat16)

You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.
Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at deepseek-ai/DeepSeek-OCR and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
# prompt = "<image>\nFree OCR. "
prompt = "<image>\n<|grounding|>Convert the document to markdown. "
image_file = 'example1.png'
output_path = 'output'

In [65]:
# infer(self, tokenizer, prompt='', image_file='', output_path = ' ', base_size = 1024, image_size = 640, crop_mode = True, test_compress = False, save_results = False):

# Tiny: base_size = 512, image_size = 512, crop_mode = False
# Small: base_size = 640, image_size = 640, crop_mode = False
# Base: base_size = 1024, image_size = 1024, crop_mode = False
# Large: base_size = 1280, image_size = 1280, crop_mode = False

# Gundam: base_size = 1024, image_size = 640, crop_mode = True

In [66]:
res = model.infer(tokenizer, prompt=prompt, image_file=image_file, output_path = output_path, base_size = 1024, image_size = 640, crop_mode=True, save_results = True, test_compress = True)

/opt/tljh/user/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


BASE:  torch.Size([1, 256, 1280])
PATCHES:  torch.Size([4, 100, 1280])
<|ref|>text<|/ref|><|det|>[[58, 145, 390, 165]]<|/det|>
Dodanie produktu do koszyka nie oznacza jego rezerwacji 

<|ref|>text<|/ref|><|det|>[[37, 210, 214, 230]]<|/det|>
Liczba produktów w koszyku: 1 

<|ref|>image<|/ref|><|det|>[[88, 300, 179, 430]]<|/det|>
 

<|ref|>sub_title<|/ref|><|det|>[[244, 255, 366, 280]]<|/det|>
Galaxy S25 Edge 

<|ref|>text<|/ref|><|det|>[[244, 297, 360, 315]]<|/det|>
Tytanowy Srebrny, 512 GB 

<|ref|>text<|/ref|><|det|>[[244, 322, 325, 338]]<|/det|>
SM-S937BZSGEUE 

<|ref|>text<|/ref|><|det|>[[244, 346, 315, 362]]<|/det|>
Przedsprzedaż 

<|ref|>text<|/ref|><|det|>[[244, 369, 417, 386]]<|/det|>
Wysyłka od: 26 maj 2025 do 13 cze 2025 

<|ref|>text<|/ref|><|det|>[[520, 255, 600, 280]]<|/det|>
4 399,00 zł 

<|ref|>text<|/ref|><|det|>[[480, 280, 600, 310]]<|/det|>
Najniższa cena z 30 dni
przed obniżką: 5 899,00 zł 

<|ref|>text<|/ref|><|det|>[[500, 308, 600, 323]]<|/det|>
Rabat 1 500,00 zł 



other: 100%|██████████| 36/36 [00:00<00:00, 137393.03it/s]


In [5]:
model.base_model

DeepseekOCRModel(
  (embed_tokens): Embedding(129280, 1280)
  (layers): ModuleList(
    (0): DeepseekV2DecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_features=1280, out_features=1280, bias=False)
        (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=False)
        (o_proj): Linear(in_features=1280, out_features=1280, bias=False)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (mlp): DeepseekV2MLP(
        (gate_proj): Linear(in_features=1280, out_features=6848, bias=False)
        (up_proj): Linear(in_features=1280, out_features=6848, bias=False)
        (down_proj): Linear(in_features=6848, out_features=1280, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): DeepseekV2RMSNorm()
      (post_attention_layernorm): DeepseekV2RMSNorm()
    )
    (1-11): 11 x DeepseekV2DecoderLayer(
      (self_attn): LlamaAttention(
        (q_proj): Linear(in_

In [35]:
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

In [36]:

@torch.no_grad()
def deepencoder_tokens_via_clip(model, png_path, device=None):
    """
    Implements: SAM-base -> 2-layer conv compressor (already inside sam_model.neck/net_2/net_3)
                 -> flatten to tokens -> add CLIP pos-emb (no CLS)
                 -> CLIP pre-layernorm + transformer
    Returns: tokens_clip  [B, N, 1024]
    """
    if device is None:
        device = next(model.parameters()).device

    img = Image.open(png_path).convert("RGB")
    x = _preproc_1024(img).unsqueeze(0).to(torch.bfloat16).to(device)  # [1,3,1024,1024] -> N=256
    
    model.eval()

    # --- SAM-base encoder + 16× token compressor ---
    # Your ImageEncoderViT already contains: patch_embed (stride 16) + blocks + neck + net_2 (stride 2) + net_3 (stride 2).
    # Its output tensor should be [B, 1024, Hs, Ws] with Hs*Ws = N.
    sam_out = model.base_model.sam_model(x)
    B, C, Hs, Ws = sam_out.shape
    assert C == 1024, f"Expect 1024 channels before CLIP, got {C}"

    # Flatten to tokens [B, N, 1024], N = Hs*Ws = (H/16/4)*(W/16/4) = HW/4096
    tokens = sam_out.flatten(2).transpose(1, 2).contiguous()  # [B,N,1024]
    N = tokens.size(1)

    # --- Add CLIP positional embeddings (no CLS token) ---
    # CLIP-large position table in your model: Embedding(257, 1024). Index 0 is CLS; 1..256 are 16×16 patches.
    pos_table = model.base_model.vision_model.embeddings.position_embedding.weight  # [257,1024]
    # Build a 2D 16×16 grid from indices 1..256, then interpolate to (Hs, Ws) if needed.
    base_grid = pos_table[1:257].view(16, 16, 1024).permute(2, 0, 1).unsqueeze(0)  # [1,1024,16,16]

    # below is optional as Hs and Ws should in 16 and 16
    # grid = F.interpolate(base_grid, size=(Hs, Ws), mode="bicubic", align_corners=False)  # [1,1024,Hs,Ws]
    
    pos = base_grid.flatten(2).transpose(1, 2)  # [1,N,1024] # base grid or grid if interpoaltion is needed
    tokens = tokens + pos # or pos

    # --- Run CLIP without its patch embed ---
    x_tok = model.base_model.vision_model.pre_layrnorm(tokens)          # [B,N,1024]
    x_tok = model.base_model.vision_model.transformer(x_tok)            # [B,N,1024] (NoTPTransformer)
    # if isinstance(x_tok, (list, tuple)):  # some impls return (x, ...)
    #     x_tok = x_tok[0]
    tokens_clip = x_tok                                                   # [B,N,1024]

    # Last step add sam out and CLIP out
    out = tokens + tokens_clip
    return out

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import math
from typing import Optional, Union
from PIL import Image
import torch
import torch.nn.functional as F
from torchvision import transforms

# --- Runtime knobs (safe speedups on Ampere+ with BF16) ---
torch.backends.cuda.matmul.allow_tf32 = True         # allow TF32 on matmul
torch.backends.cudnn.allow_tf32 = True               # allow TF32 on cudnn
torch.backends.cudnn.benchmark = True                # autotune convs
# Prefer Flash SDP if your PyTorch build supports it (falls back automatically)
try:
    torch.backends.cuda.enable_flash_sdp(True)
    torch.backends.cuda.enable_mem_efficient_sdp(False)
    torch.backends.cuda.enable_math_sdp(False)
except Exception:
    pass


class DeepSeekOCREncoder(torch.nn.Module):
    """
    Optimized, memory-lean encoder:
      SAM-base (with built-in conv compressor) -> [B,1024,Hs,Ws]
      flatten -> add CLIP 2D pos-emb (no CLS)
      -> CLIP pre-layernorm + transformer
      returns tokens + CLIP(tokens)  [B, N, 1024]
    """

    def __init__(
        self,
        full_model,
        device: Optional[Union[str, torch.device]] = None,
        dtype: torch.dtype = torch.bfloat16,
        freeze: bool = True,
        eager_to_device: bool = True,
        precompute_pos_for_1024: bool = True,
        use_compile: bool = False,
    ):
        super().__init__()

        if device is None:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.device = torch.device(device)
        self.dtype = dtype
        self.embed_dim = 1024

        base = getattr(full_model, "base_model", full_model)
        self.sam = getattr(base, "sam_model").eval()
        vision = getattr(base, "vision_model")
        self.clip_pre = vision.pre_layrnorm.eval()
        self.clip_tr = vision.transformer.eval()

        # Position table (register as buffer so it moves with .to())
        pos_table = vision.embeddings.position_embedding.weight.detach()  # [1+G^2, 1024]
        self.register_buffer("clip_pos_table", pos_table, persistent=False)

        # Drop likely-unused heavy parts to save RAM/VRAM
        def _safe_del(obj, name):
            if hasattr(obj, name):
                try:
                    setattr(obj, name, None)
                except Exception:
                    pass

        _safe_del(vision.embeddings, "patch_embedding")
        _safe_del(vision, "post_layernorm")
        _safe_del(base, "text_model")
        _safe_del(base, "lm_head")
        _safe_del(full_model, "text_model")
        _safe_del(full_model, "lm_head")

        # If you want N=256 like the paper figure, feed 1024×1024.
        self._preproc_1024 = transforms.Compose([
            transforms.Resize((1024, 1024), interpolation=transforms.InterpolationMode.BICUBIC, antialias=True),
            transforms.ToTensor(),
            # Use the SAM/CLIP-normalization your training used. If unsure, CLIP's works well in practice:
            transforms.Normalize(mean=(0.48145466, 0.4578275, 0.40821073),
                                 std=(0.26862954, 0.26130258, 0.27577711)),
        ])

        if freeze:
            for m in [self.sam, self.clip_pre, self.clip_tr]:
                for p in m.parameters():
                    p.requires_grad_(False)

        if eager_to_device:
            # channels_last helps conv-heavy encoders; safe for BF16
            self.sam.to(self.device, dtype=self.dtype, memory_format=torch.channels_last)
            self.clip_pre.to(self.device, dtype=self.dtype)
            self.clip_tr.to(self.device, dtype=self.dtype)
            self.clip_pos_table.data = self.clip_pos_table.to(self.device, dtype=self.dtype)

        # Precompute pos-emb for the common 1024→N=256 case (Hs=Ws=16)
        self._pos_cache = {}
        self._pos_fixed_16 = None
        if precompute_pos_for_1024:
            table = self.clip_pos_table
            grid_size = int(math.isqrt(table.size(0) - 1))
            if grid_size * grid_size + 1 != table.size(0):
                raise RuntimeError("Position table size is not 1+G^2; cannot infer grid size.")
            base_grid = table[1: 1 + grid_size * grid_size]  # [G^2, 1024]
            base_grid = base_grid.view(grid_size, grid_size, self.embed_dim).permute(2, 0, 1).unsqueeze(0)  # [1,1024,G,G]
            if grid_size != 16:
                base_grid = F.interpolate(base_grid, size=(16, 16), mode="bicubic", align_corners=False)
            self._pos_fixed_16 = base_grid.flatten(2).transpose(1, 2).contiguous()  # [1,256,1024]

        # Optional torch.compile for better kernel fusion on the transformer path
        self._compiled = None
        if use_compile and hasattr(torch, "compile"):
            self._compiled = torch.compile(self._forward_core, mode="max-autotune")  # PyTorch 2.3+

        # Optional CUDA Graphs (opt-in; call capture_cudagraph() once before steady-state)
        self._graph = None
        self._static_in = None
        self._static_out = None

    @torch.inference_mode()
    def _forward_core(self, x_nchw_channels_last: torch.Tensor) -> torch.Tensor:
        """
        x_nchw_channels_last: [B,3,1024,1024] BF16 on self.device, channels_last
        """
        sam_out = self.sam(x_nchw_channels_last)  # [B, 1024, Hs, Ws]
        B, C, Hs, Ws = sam_out.shape
        if C != self.embed_dim:
            raise RuntimeError(f"Expected {self.embed_dim} channels before CLIP, got {C}")

        tokens = sam_out.flatten(2).transpose(1, 2).contiguous()  # [B, N, 1024], N=Hs*Ws

        if Hs == 16 and Ws == 16 and self._pos_fixed_16 is not None:
            pos = self._pos_fixed_16  # [1,256,1024]
        else:
            key = (Hs, Ws)
            pos = self._pos_cache.get(key)
            if pos is None:
                table = self.clip_pos_table
                grid_size = int(math.isqrt(table.size(0) - 1))
                base_grid = table[1: 1 + grid_size * grid_size].view(grid_size, grid_size, self.embed_dim)
                base_grid = base_grid.permute(2, 0, 1).unsqueeze(0)  # [1,1024,G,G]
                if (Hs, Ws) != (grid_size, grid_size):
                    base_grid = F.interpolate(base_grid, size=(Hs, Ws), mode="bicubic", align_corners=False)
                pos = base_grid.flatten(2).transpose(1, 2).contiguous()  # [1,N,1024]
                self._pos_cache[key] = pos

        tokens_plus = tokens + pos  # broadcast over batch
        x_tok = self.clip_pre(tokens_plus)
        x_tok = self.clip_tr(x_tok)
        return tokens + x_tok  # [B,N,1024]

    @torch.inference_mode()
    def encode(self, image: Union[Image.Image, str, "os.PathLike"]) -> torch.Tensor:
        """
        image can be a PIL.Image or a path-like to an RGB image.
        Returns: [1, N, 1024]  (N=256 for 1024×1024 input)
        """
        if isinstance(image, (str, bytes)):
            img = Image.open(image).convert("RGB")
        else:
            img = image.convert("RGB")

        # CPU preproc → pinned → non_blocking H2D
        x_cpu = self._preproc_1024(img).unsqueeze(0).pin_memory()                 # [1,3,1024,1024] on CPU
        x = x_cpu.to(self.device, dtype=self.dtype, non_blocking=True)       # -> GPU BF16
        x = x.to(memory_format=torch.channels_last)                          # NHWC memory layout

        # Fast path: CUDA Graph replay if captured
        if self._graph is not None:
            self._static_in.copy_(x)                                         # copy into static buffer
            self._graph.replay()
            return self._static_out

        # Compiled path if enabled; otherwise plain
        if self._compiled is not None:
            return self._compiled(x)
        else:
            return self._forward_core(x)

    @torch.inference_mode()
    def capture_cudagraph(self, batch_size: int = 1, H: int = 1024, W: int = 1024):
        """
        Capture a CUDA graph for the steady-state 1×3×1024×1024 path.
        Call once after initialization; then .encode(...) will reuse it.
        """
        if self.device.type != "cuda":
            raise RuntimeError("CUDA Graphs require CUDA device.")
        if self._graph is not None:
            return  # already captured

        # Static input/output buffers (must not resize later)
        static_in = torch.empty(
            batch_size, 3, H, W, device=self.device, dtype=self.dtype
        ).to(memory_format=torch.channels_last)
        # Warm-up to materialize kernels & memory pools
        for _ in range(3):
            _ = self._forward_core(static_in)

        g = torch.cuda.CUDAGraph()
        torch.cuda.synchronize()
        with torch.cuda.graph(g):
            static_out = self._forward_core(static_in)
        self._graph = g
        self._static_in = static_in
        self._static_out = static_out


In [3]:
# Example

In [4]:
from transformers import AutoModel
import torch

In [5]:
model_name = "deepseek-ai/DeepSeek-OCR"

# If your env supports it, prefer SDPA/Flash attention over eager.
# In recent transformers the arg is `attn_implementation`; your underscore version also works for some models.
attn_kw = dict(attn_implementation="eager")  # or "flash_attention_2" if installed; else keep "eager"

model = AutoModel.from_pretrained(
    model_name,
    trust_remote_code=True,
    use_safetensors=True,
    torch_dtype=torch.bfloat16,
    **attn_kw,
)

model = model.eval().to("cuda", dtype=torch.bfloat16)

You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.
Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at deepseek-ai/DeepSeek-OCR and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Wrap the lean encoder
enc = DeepSeekOCREncoder(
    full_model=model,
    device="cuda",
    dtype=torch.bfloat16,
    freeze=True,
    eager_to_device=True,
    precompute_pos_for_1024=True,
    use_compile=False,  # set True if you're on PyTorch 2.3+ and want extra fusion
)

In [7]:
# Optional: capture a CUDA Graph for 1×3×1024×1024 steady-state inference
enc.capture_cudagraph(batch_size=1, H=1024, W=1024)

In [8]:
image_file = 'example1.png'
img = Image.open(image_file).convert("RGB")

In [10]:
%timeit vision_tokens = enc.encode(img)

The slowest run took 4.08 times longer than the fastest. This could mean that an intermediate result is being cached.
58.7 ms ± 40.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
vision_tokens.shape

torch.Size([1, 256, 1024])